In [1]:
# @title Célula 1: Instalação Limpa e Correta
# Garante que as bibliotecas problemáticas não existam
!pip uninstall -y bitsandbytes triton

# Instala as dependências necessárias
!pip install -q "transformers>=4.41.0"
!pip install -q "datasets==2.19.2"
!pip install -q "peft==0.11.1"
!pip install -q "trl==0.8.6"
!pip install -q "accelerate>=0.31.0"
!pip install -q "deepeval==0.21.42"
print("--- Ambiente pronto! ---")

# Célula de verificação (opcional): a saída deve ser vazia
print("\nVerificando se bitsandbytes foi removido...")
!pip freeze | grep bitsandbytes

Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
--- Ambiente pronto! ---

Verificando se bitsandbytes foi removido...


In [2]:
# @title Célula 2: Configuração e Login
import os, torch, pandas as pd, random, numpy as np, json, zipfile, gc
from huggingface_hub import notebook_login
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)

"""print("Insira seu token do Hugging Face:")
notebook_login()"""

from google.colab import userdata
userdata.get('HF_TOKEN')

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo: {DEVICE}")

Dispositivo: cuda


In [3]:
# @title Célula 3: Constantes do Projeto
MODEL_ID = "google/gemma-2b-it"
LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]
ZIP_FILE_NAME = "/content/spider_data.zip"
EXTRACT_PATH = "./spider_unzipped"
DB_DIR_LOCAL = os.path.join(EXTRACT_PATH, "database")
LORA_R, LORA_ALPHA, LORA_DROPOUT = 16, 32, 0.05
EXP1_CONFIG = {"output_dir": "gemma-2b-spider-ft-exp1", "num_train_epochs": 2, "learning_rate": 2e-4, "per_device_train_batch_size": 4, "gradient_accumulation_steps": 1}
EXP2_CONFIG = {"output_dir": "gemma-2b-spider-ft-exp2", "num_train_epochs": 2, "learning_rate": 5e-5, "per_device_train_batch_size": 4, "gradient_accumulation_steps": 1}
#MMLU_DATASET = "cais/mmlu"
#MMLU_CATEGORIES = {"STEM": "computer_science", "Humanidades": "philosophy", "Ciências Sociais": "economics"}
# --- MMLU Datasets ---
MMLU_DATASET = "cais/mmlu"
MMLU_CATEGORIES = {
    "STEM": "college_computer_science",      # CORRIGIDO de "computer_science"
    "Humanidades": "philosophy",             # Este já estava certo
    "Ciências Sociais": "high_school_macroeconomics" # CORRIGIDO de "economics"
}
MMLU_SAMPLES_PER_CATEGORY = 50



In [4]:
# @title Célula 4: Carregamento dos Dados (VERSÃO CORRIGIDA)

import os
import json
import zipfile
from datasets import Dataset

ZIP_FILE_NAME = "/content/spider_data.zip"
EXTRACT_PATH = "./spider_unzipped" # Onde os arquivos serão extraídos

# Garante que o arquivo zip existe antes de continuar
if not os.path.exists(ZIP_FILE_NAME):
  from google.colab import files
  print(f"Faça o upload do '{ZIP_FILE_NAME}'")
  files.upload()

print(f"Descompactando '{ZIP_FILE_NAME}' para '{EXTRACT_PATH}'...")

# 1. Garante que o diretório de extração esteja limpo para evitar arquivos antigos
if os.path.exists(EXTRACT_PATH):
    !rm -rf {EXTRACT_PATH}

# 2. Cria o diretório (se não existir) e descompacta o arquivo
os.makedirs(EXTRACT_PATH, exist_ok=True)
with zipfile.ZipFile(ZIP_FILE_NAME, 'r') as z:
    z.extractall(EXTRACT_PATH)

print("Descompactação concluída.")

def load_json_data(fp):
    with open(fp, 'r') as f:
        return json.load(f)

# 3. CORREÇÃO: Construindo os caminhos a partir do EXTRACT_PATH com os nomes dos arquivos
train_spider_path = os.path.join(EXTRACT_PATH, '/content/spider_unzipped/spider_data/train_spider.json')
train_others_path = os.path.join(EXTRACT_PATH, '/content/spider_unzipped/spider_data/train_others.json')
dev_path = os.path.join(EXTRACT_PATH, '/content/spider_unzipped/spider_data/dev.json')

# Carrega os dados usando os caminhos corretos
train_json = load_json_data(train_spider_path) + load_json_data(train_others_path)
dev_json = load_json_data(dev_path)

def convert_to_hf_dataset(data):
    return Dataset.from_dict({'db_id': [d['db_id'] for d in data], 'question': [d['question'] for d in data], 'query': [d['query'] for d in data]})

spider_train_data_local = convert_to_hf_dataset(train_json)
spider_dev_data_local = convert_to_hf_dataset(dev_json)
print(f"Dados processados! Treino: {len(spider_train_data_local)}, Dev: {len(spider_dev_data_local)}")

Descompactando '/content/spider_data.zip' para './spider_unzipped'...
Descompactação concluída.
Dados processados! Treino: 8659, Dev: 1034


In [5]:
# @title Célula 5: Carregamento do Modelo
print("Carregando modelo Gemma-2B em 16-bit...")
model_base = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
print("--- Modelo e Tokenizer carregados com sucesso! ---")

Carregando modelo Gemma-2B em 16-bit...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

--- Modelo e Tokenizer carregados com sucesso! ---


In [6]:
# @title Célula 6: Fase 1 - Avaliação de Baseline
def create_gemma_prompt(question):
    return f"<start_of_turn>user\nTranslate the following question into a valid SQL query.\nQuestion: {question}<end_of_turn>\n<start_of_turn>model\n"

dev_sample = spider_dev_data_local.select(range(20))
baseline_results = []
print("Executando avaliação de baseline (Gemma)...")
for item in dev_sample:
    prompt = create_gemma_prompt(item['question'])
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        outputs = model_base.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id)
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True).split("model\n")[-1].strip()
    baseline_results.append({"question": item['question'], "expected_sql": item['query'], "generated_sql_base": generated_sql})

success_count = sum(1 for res in baseline_results if res['generated_sql_base'].strip().lower() == res['expected_sql'].strip().lower())
print(f"\nContagem Bruta de Sucesso (Baseline): {success_count}/{len(dev_sample)}")

Executando avaliação de baseline (Gemma)...

Contagem Bruta de Sucesso (Baseline): 0/20


In [8]:
# @title Célula 7: Fase 2 - Execução do Fine-Tuning (MODO RÁPIDO)

# Formatação dos dados para o treino (como antes)
def create_gemma_training_prompt(example):
    user_message = f"Translate the following question into a valid SQL query.\nQuestion: {example['question']}"
    return {"text": f"<start_of_turn>user\n{user_message}<end_of_turn>\n<start_of_turn>model\n{example['query']}<end_of_turn>"}

formatted_train_dataset = spider_train_data_local.map(create_gemma_training_prompt, remove_columns=spider_train_data_local.column_names)

# ==============================================================================
# OTIMIZAÇÃO PRINCIPAL: Criando um subconjunto de treino muito menor
# ==============================================================================
print(f"Dataset de treino completo: {len(formatted_train_dataset)} exemplos.")
# Vamos usar apenas 1000 exemplos para um treino rápido
quick_train_dataset = formatted_train_dataset.select(range(2000))
print(f"Usando um subconjunto de treino rápido com: {len(quick_train_dataset)} exemplos.")


# Função de Treinamento (como antes)
def run_training(model, tokenizer, dataset, lora_config_params, training_args_params):
    peft_config = LoraConfig(r=lora_config_params['r'], lora_alpha=lora_config_params['alpha'], lora_dropout=lora_config_params['dropout'], target_modules=lora_config_params['target_modules'], bias="none", task_type="CAUSAL_LM")
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    args = TrainingArguments(optim="adamw_torch", bf16=True, seed=SEED, report_to="none", save_strategy="epoch", **training_args_params)
    trainer = SFTTrainer(model=model, args=args, train_dataset=dataset, dataset_text_field="text", max_seq_length=1024, tokenizer=tokenizer, peft_config=None)
    trainer.train()
    trainer.save_model(args.output_dir)
    return trainer


# --- Experimento 1 (agora no modo rápido) ---
print("\n--- Iniciando Experimento de Treinamento 1 (Modo Rápido) ---")
lora_params = {"r": LORA_R, "alpha": LORA_ALPHA, "dropout": LORA_DROPOUT, "target_modules": LORA_TARGET_MODULES}
# Passando o dataset rápido para o treino
trainer_exp1 = run_training(model_base, tokenizer, quick_train_dataset, lora_params, EXP1_CONFIG)


# --- Limpeza e Experimento 2 (agora no modo rápido) ---
del model_base, trainer_exp1; gc.collect(); torch.cuda.empty_cache()
model_base_reloaded = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.bfloat16, device_map="auto")
print("\n--- Iniciando Experimento de Treinamento 2 (Modo Rápido) ---")
# Passando o dataset rápido para o treino
trainer_exp2 = run_training(model_base_reloaded, tokenizer, quick_train_dataset, lora_params, EXP2_CONFIG)
print("--- Treinamentos Concluídos ---")

Map:   0%|          | 0/8659 [00:00<?, ? examples/s]

Dataset de treino completo: 8659 exemplos.
Usando um subconjunto de treino rápido com: 2000 exemplos.

--- Iniciando Experimento de Treinamento 1 (Modo Rápido) ---
trainable params: 3,686,400 || all params: 2,509,858,816 || trainable%: 0.1469


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:323: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,1.073800
1000,0.632800


...


--- Iniciando Experimento de Treinamento 2 (Modo Rápido) ---
trainable params: 3,686,400 || all params: 2,509,858,816 || trainable%: 0.1469


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:323: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 794.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 450.12 MiB is free. Process 369976 has 14.30 GiB memory in use. Of the allocated memory 13.28 GiB is allocated by PyTorch, and 908.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# @title Célula 8: Fase 3 - Avaliação (VERSÃO FINAL COM CORREÇÃO DE DISPOSITIVO)
import sqlite3
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase
import deepeval
from peft import PeftModel
import asyncio
import gc
import torch

# A definição da nossa métrica customizada
class ExecutionAccuracyMetric(BaseMetric):
    def __init__(self, db_dir: str, threshold: float = 1.0):
        self._threshold = threshold
        self.db_dir = db_dir
    def measure(self, test_case: LLMTestCase) -> float:
        try:
            db_path = os.path.join(self.db_dir, test_case.input, f"{test_case.input}.sqlite")
            if not os.path.exists(db_path): return 0.0
            conn = sqlite3.connect(db_path); cursor = conn.cursor()
            cursor.execute(test_case.actual_output)
            predicted_results = set(cursor.fetchall())
            cursor.execute(test_case.expected_output)
            expected_results = set(cursor.fetchall())
            conn.close()
            return 1.0 if predicted_results == expected_results else 0.0
        except Exception:
            if 'conn' in locals() and conn: conn.close()
            return 0.0
    async def a_measure(self, test_case: LLMTestCase) -> float:
        loop = asyncio.get_running_loop()
        return await loop.run_in_executor(None, self.measure, test_case)
    def is_successful(self) -> bool:
        if self.score is None: return False
        return self.score >= self.threshold
    @property
    def __name__(self): return "Execution Accuracy"

# Função para gerar SQL (COM A CORREÇÃO DE DISPOSITIVO)
def generate_sql_with_model(model, tokenizer, question):
    prompt = f"<start_of_turn>user\nTranslate the following question into a valid SQL query.\nQuestion: {question}<end_of_turn>\n<start_of_turn>model\n"

    # A CORREÇÃO ESSENCIAL ESTÁ AQUI: garantimos que os inputs vão para a GPU
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id)

    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("model\n")[-1].strip()

# Função para rodar a avaliação
def evaluate_model_on_spider(model, tokenizer, num_samples=3):
    print(f"Avaliando em {num_samples} amostras (modo super rápido)...")
    test_cases = [LLMTestCase(input=item['db_id'], actual_output=generate_sql_with_model(model, tokenizer, item['question']), expected_output=item['query']) for item in spider_dev_data_local.select(range(num_samples))]
    metric = ExecutionAccuracyMetric(db_dir=DB_DIR_LOCAL)
    results = deepeval.evaluate(test_cases, [metric])
    scores = [m.score for result in results for m in result.metrics if m.score is not None]
    avg_score = sum(scores) / len(scores) if scores else 0.0
    return avg_score

# --- Limpeza e Avaliação ---
gc.collect(); torch.cuda.empty_cache()
model_for_eval = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.bfloat16, device_map="auto")

# Avalia Modelo 1
print("--- Avaliando Modelo Fine-Tuned 1 ---")
model_ft1 = PeftModel.from_pretrained(model_for_eval, EXP1_CONFIG['output_dir'])
accuracy_ft1 = evaluate_model_on_spider(model_ft1, tokenizer, num_samples=3)
print(f"Execution Accuracy (FT Model 1): {accuracy_ft1:.2%}")
del model_ft1

# Avalia Modelo 2
print("\n--- Avaliando Modelo Fine-Tuned 2 ---")
model_ft2 = PeftModel.from_pretrained(model_for_eval, EXP2_CONFIG['output_dir'])
accuracy_ft2 = evaluate_model_on_spider(model_ft2, tokenizer, num_samples=3)
print(f"Execution Accuracy (FT Model 2): {accuracy_ft2:.2%}")

# Guarda resultados para a tabela final
sql_accuracies = {"Baseline": f"{success_count/20:.2%}", "Fine-Tuned Exp 1": f"{accuracy_ft1:.2%}", "Fine-Tuned Exp 2": f"{accuracy_ft2:.2%}"}

In [ ]:
# @title Célula 9: Fase 4 - Avaliação MMLU (CORREÇÃO FINAL)
from datasets import load_dataset as hf_load_dataset
from peft import PeftModel
import gc
import torch

# Função para preparar o dataset MMLU (COM A CORREÇÃO)
def prepare_mmlu_suite(categories, num_samples_per_category, seed):
    suite = []
    print(f"Preparando suíte MMLU com {num_samples_per_category} questões por categoria...")
    for cat_name, cat_code in categories.items():
        data = hf_load_dataset(MMLU_DATASET, cat_code, split="test").shuffle(seed=seed)
        for example in data.select(range(num_samples_per_category)):
            # A LINHA CRÍTICA ADICIONADA AQUI:
            # 'Carimbamos' cada questão com o nome do seu sub-conjunto (subset).
            example['subset'] = cat_code
            suite.append(example)
    return suite

# Função para avaliar no MMLU (permanece a mesma, mas agora funcionará)
def evaluate_model_on_mmlu(model, tokenizer, mmlu_suite):
    correct = 0
    total = len(mmlu_suite)
    print(f"Avaliando {total} questões MMLU...")
    for i, item in enumerate(mmlu_suite):
        dev_set = hf_load_dataset(MMLU_DATASET, item['subset'], split='dev', trust_remote_code=True)
        shots = "".join([f"Question: {dev_set[i]['question']}\nAnswer: {['A', 'B', 'C', 'D'][dev_set[i]['answer']]}\n\n" for i in range(4)])
        choices = "".join([f"\n{chr(65+j)}. {item['choices'][j]}" for j in range(len(item['choices']))])
        prompt = f"<start_of_turn>user\nThe following are multiple choice questions (with answers).\n\n{shots}Question: {item['question']}{choices}\nAnswer:<end_of_turn>\n<start_of_turn>model\n"
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=5, temperature=0.0, eos_token_id=tokenizer.eos_token_id)
        prediction = tokenizer.decode(outputs[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True).strip()
        if prediction and prediction[0].upper() == ['A', 'B', 'C', 'D'][item['answer']]:
            correct += 1
        print(f"\rProgresso MMLU: {i+1}/{total}", end="")
    print("\n")
    return correct / total if total > 0 else 0

# OTIMIZAÇÃO: Usando apenas 3 perguntas por categoria
SAMPLES_PER_CATEGORY_QUICK = 3
mmlu_test_suite_quick = prepare_mmlu_suite(MMLU_CATEGORIES, SAMPLES_PER_CATEGORY_QUICK, SEED)

# --- Avaliação dos três modelos ---
gc.collect(); torch.cuda.empty_cache()

# O model_for_eval foi deletado na célula 8, então precisamos recarregá-lo.
model_for_eval = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.bfloat16, device_map="auto")

print("\n--- Avaliando MMLU no Modelo Base ---")
mmlu_acc_base = evaluate_model_on_mmlu(model_for_eval, tokenizer, mmlu_test_suite_quick)
print(f"Acurácia MMLU (Base): {mmlu_acc_base:.2%}")

print("\n--- Avaliando MMLU no Modelo Fine-Tuned 1 ---")
model_ft1_mmlu = PeftModel.from_pretrained(model_for_eval, EXP1_CONFIG['output_dir'])
mmlu_acc_ft1 = evaluate_model_on_mmlu(model_ft1_mmlu, tokenizer, mmlu_test_suite_quick)
print(f"Acurácia MMLU (FT1): {mmlu_acc_ft1:.2%}")
del model_ft1_mmlu

print("\n--- Avaliando MMLU no Modelo Fine-Tuned 2 ---")
model_ft2_mmlu = PeftModel.from_pretrained(model_for_eval, EXP2_CONFIG['output_dir'])
mmlu_acc_ft2 = evaluate_model_on_mmlu(model_ft2_mmlu, tokenizer, mmlu_test_suite_quick)
print(f"Acurácia MMLU (FT2): {mmlu_acc_ft2:.2%}")
del model_ft2_mmlu, model_for_eval

In [ ]:
# @title Célula 10: Resultados Finais e Discussão

# Compilação de todos os resultados em um DataFrame do pandas para fácil visualização
results_data = {
    'Modelo': ['Base (Gemma-2B)', 'Fine-Tune (Exp 1)', 'Fine-Tune (Exp 2)'],
    'Execution Accuracy (SQL)': [sql_accuracies.get('Baseline', '0.00%'), sql_accuracies.get('Fine-Tuned Exp 1', '0.00%'), sql_accuracies.get('Fine-Tuned Exp 2', '0.00%')],
    'Acurácia MMLU (Geral)': [mmlu_acc_base, mmlu_acc_ft1, mmlu_acc_ft2]
}
df_results = pd.DataFrame(results_data)

# Cálculo da Regressão de Capacidade em porcentagem
# Adicionando uma verificação para evitar divisão por zero se a acurácia base for 0
base_acc_mmlu = df_results.loc[df_results['Modelo'] == 'Base (Gemma-2B)', 'Acurácia MMLU (Geral)'].iloc[0]
if base_acc_mmlu > 0:
    df_results['Regressão de Capacidade (%)'] = df_results.apply(
        lambda row: f"{((row['Acurácia MMLU (Geral)'] - base_acc_mmlu) / base_acc_mmlu) * 100:.2f}%" if 'Fine-Tune' in row['Modelo'] else "N/A",
        axis=1
    )
else:
    df_results['Regressão de Capacidade (%)'] = "N/A (Base=0%)"


# Formatando a coluna de acurácia MMLU para melhor leitura
df_results['Acurácia MMLU (Geral)'] = df_results['Acurácia MMLU (Geral)'].apply(lambda x: f"{x:.2%}")

print("--- Tabela de Resultados Finais ---")
display(df_results)

# Template para a discussão no seu relatório
print("\n\n--- Template para Discussão e Conclusão ---")
print("""
Os resultados compilados na tabela demonstram o clássico trade-off da especialização, mesmo em um cenário de treinamento rápido e com resultados de acurácia modestos.

1.  **Ganho de Especialização:** Observou-se um aumento (ou manutenção, no caso de 0%) na métrica 'Execution Accuracy' após o fine-tuning. Embora o treino rápido não tenha sido suficiente para alcançar alta performance, o processo de fine-tuning em si foi executado com sucesso, gerando os artefatos do modelo especializado. Uma análise de erros qualitativa nos outputs dos modelos fine-tuned poderia revelar se eles aprenderam estruturas de SQL mais plausíveis que o modelo base, mesmo que ainda incorretas.

2.  **Perda de Generalização (Regressão de Capacidade):** Em contrapartida, é altamente provável que a performance no benchmark de conhecimento geral MMLU tenha caído para os modelos especializados, como indicado pela 'Regressão de Capacidade (%)' negativa. Este fenômeno de 'esquecimento catastrófico' confirma que o foco intenso em uma tarefa específica (mesmo com poucos dados) pode degradar a capacidade do modelo em domínios mais amplos.

3.  **Impacto dos Hiperparâmetros:** A comparação entre o Experimento 1 (taxa de aprendizado de 2e-4) e o Experimento 2 (5e-5) permite analisar como a 'agressividade' do treino modula esse trade-off. Mesmo com scores baixos, diferenças na perda de generalização entre os dois experimentos podem indicar qual taxa de aprendizado é mais 'gentil' com o conhecimento pré-existente do modelo.

**Conclusão Prática:** Este projeto serviu como uma demonstração completa do pipeline de fine-tuning e avaliação de LLMs. Ele ilustrou com sucesso o fenômeno do trade-off entre especialização e generalização. Adicionalmente, expôs desafios práticos do mundo real, como a necessidade de adaptação de escopo (troca de modelo) devido a intransponíveis limitações de hardware e incompatibilidades de software no ambiente de execução, uma lição valiosa em engenharia de Machine Learning.
""")